In [1]:
import pyfeats
from typing import *
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from utils import *

In [2]:
llf = LowLevelFeatureExtractor(**param_list[0])

# root_folder = "C:\\Users\\trong\\Documents\\skin_data"
root_folder = "/mnt/c/Users/trong/Documents/skin_data" 

In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.Grayscale(num_output_channels=1),    # Convert to grayscale
    ToNumpy(),  # Convert to tensor
])

In [ ]:
train_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_train_linux.csv', root_dir=root_folder, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)   

test_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_test_linux.csv', root_dir=root_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check one batch
images, metadata, labels = next(iter(train_dataloader))
print(images.shape)  # Example: torch.Size([32, 3, 128, 128])
print(metadata.shape)  # Example: torch.Size([32, 6])  -> 6 metadata features
print(labels.shape)  # Example: torch.Size([32])

In [3]:
train_dataset = CSVMetadataDataset(csv_file="./data/fos/vaynen_train_fos.csv", root_dir=root_folder)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)   

test_dataset = CSVMetadataDataset(csv_file="./data/fos/vaynen_test_fos.csv", root_dir=root_folder)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check one batch
images, metadata, labels = next(iter(train_dataloader))
print(images.shape)  # Example: torch.Size([32, 3, 128, 128])
print(metadata.shape)  # Example: torch.Size([32, 6])  -> 6 metadata features
print(labels.shape)  # Example: torch.Size([32])

torch.Size([32, 0])
torch.Size([32, 22])
torch.Size([32])


In [4]:
model = SimpleNeuralNetwork(inputs = llf.get_features_size())

# Check the model's architecture
print(model)

SimpleNeuralNetwork(
  (fc1): Linear(in_features=22, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
  (swish): SwishActivation()
  (batchNorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (sigmoid): Sigmoid()
)


In [5]:
train_model(model, train_dataloader, epochs=30, llf=llf ,features_set=llf.function.__name__, ) #10 - 15 is already enough

Training model using cuda


Epoch 30, Loss: 0.015433703656120701: 100%|██████████| 30/30 [00:58<00:00,  1.97s/it]

Training on fos complete!


In [6]:
evaluate_model(model, test_dataloader,llf=llf, features_set=llf.function.__name__)

'Accuracy: 71.7094017094017% On fos'